In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import pickle

reducers = ee.Reducer.mean().combine(**
                                     {'reducer2': ee.Reducer.stdDev(), 
                                      'sharedInputs': True}).combine(**
                                                                     {'reducer2': ee.Reducer.max(), 
                                                                      'sharedInputs': True}).combine(**
                                                                                                     {'reducer2':ee.Reducer.min(), 
                                                                                                      'sharedInputs':True}).combine(**
                                                                                                                                    {'reducer2': ee.Reducer.percentile([25, 50, 75]), 
                                                                                                                                     'sharedInputs':True})
def det(im):
    return im.expression('b(0) * b(1)')
    
def chi2cdf(chi2, df):
    ''' Chi square cumulative distribution function for df degrees of freedom
    using the built-in incomplete gamma function gammainc() '''
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def sar(before, after, geometry): 
    m = 5
    
    try:
        # The observed test statistic image -2logq
        m2logq = det(before).log().add(det(after).log()).subtract(det(before.add(after)).log().multiply(2)).add(4*np.log(2)).multiply(-2*m)
        
        # The P value image prob(m2logQ > m2logq) = 1 - prob(m2logQ < m2logq).
        p_value = ee.Image.constant(1).subtract(chi2cdf(m2logq, 2))
        
        c_map = p_value.multiply(0).where(p_value.lt(0.05), 1)
        
        diff = after.subtract(before) # Getting the difference between the two images
        d_map = c_map.multiply(0)                    # Initialize the direction map to zero.
        d_map = d_map.where(det(diff).gt(0), 1)      # All pos or neg def diffs are now labeled 1.
        d_map = d_map.where(diff.select(0).gt(0), 2) # Re-label pos def (and label some indef) to 2.
        d_map = d_map.where(det(diff).lt(0), 1)      # Label all indef to 1.
        c_map = c_map.multiply(d_map) # Re-label the c_map, 0*X = 0, 1*1 = 1, 1*2= 2, 1*3 = 3.
        
        stats = c_map.reduceRegion(**{'reducer':reducers, 'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        mean = stats['constant_mean']
        stddev = stats['constant_stdDev']
        p25 = stats['constant_p25']
        p50 = stats['constant_p50']
        p75 = stats['constant_p75']
        mini = stats['constant_min']
        maxi = stats['constant_max']
        
    except: 
        mean = None
        stddev = None
        p25 = None
        p50 = None
        p75 = None
        min = None
        max = None
        
    return p25, p50, p75, mean, stddev, mini, maxi

def nbr(before, after, geometry):
    try:
        stats_b = before.normalizedDifference(['B5', 'B7']).rename('NBR').reduceRegion(**{'reducer':reducers,
                                                                                          'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        stats_a = after.normalizedDifference(['B5', 'B7']).rename('NBR').reduceRegion(**{'reducer':reducers,
                                                                                         'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()

        mean_b = stats_b['NBR_mean']
        mean_a = stats_a['NBR_mean']

        p25_b = stats_b['NBR_p25']
        p25_a = stats_a['NBR_p25']

        p50_b = stats_b['NBR_p50']
        p50_a = stats_a['NBR_p50']

        p75_b = stats_b['NBR_p75']
        p75_a = stats_a['NBR_p75']

        stddev_b = stats_b['NBR_stdDev']
        stddev_a = stats_a['NBR_stdDev']

        min_b = stats_b['NBR_min']
        min_a = stats_a['NBR_min']

        max_b = stats_b['NBR_max']
        max_a = stats_a['NBR_max']

        p25 = p25_a - p25_b * 100
        p50 = p50_a - p50_b * 100
        p75 = p75_a - p50_b * 100
        mean = mean_a - mean_b * 100
        mini = min_a - min_b * 100
        maxi = max_a - max_b * 100
        stddev = stddev_a - stddev_b * 100
  
    except: 
        p25 = None
        p50 = None
        p75 = None
        mean = None
        stddev = None
        mini = None
        maxi = None
    
    return p25, p50, p75, mean, stddev, mini, maxi

def ndvi(before, after, geometry):
    
    try:
        stats_b = before.normalizedDifference(['B5', 'B4']).rename('NDVI').reduceRegion(**{'reducer':reducers,
                                                                                           'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        
        stats_a = after.normalizedDifference(['B5', 'B4']).rename('NDVI').reduceRegion(**{'reducer':reducers,
                                                                                          'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()

        mean_b = stats_b['NDVI_mean']
        mean_a = stats_a['NDVI_mean']

        p25_b = stats_b['NDVI_p25']
        p25_a = stats_a['NDVI_p25']

        p50_b = stats_b['NDVI_p50']
        p50_a = stats_a['NDVI_p50']

        p75_b = stats_b['NDVI_p75']
        p75_a = stats_a['NDVI_p75']

        stddev_b = stats_b['NDVI_stdDev']
        stddev_a = stats_a['NDVI_stdDev']

        min_b = stats_b['NDVI_min']
        min_a = stats_a['NDVI_min']

        max_b = stats_b['NDVI_max']
        max_a = stats_a['NDVI_max']

        p25 = p25_a - p25_b * 100
        p50 = p50_a - p50_b * 100
        p75 = p75_a - p50_b * 100
        mean = mean_a - mean_b * 100
        mini = min_a - min_b * 100
        maxi = max_a - max_b * 100
        stddev = stddev_a - stddev_b * 100 
  
    except: 
        p25 = None
        p50 = None
        p75 = None
        mean = None
        stddev = None
        mini = None
        maxi = None
    
    return p25, p50, p75, mean, stddev, mini, maxi

def evi(before, after, geometry):
    try:
        before_evi = before.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': before.select('B5'),
                'RED': before.select('B4'),
                'BLUE': before.select('B2')})

        after_evi = after.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': after.select('B5'),
                'RED': after.select('B4'),
                'BLUE': after.select('B2')})

        stats_b = before_evi.rename('EVI').reduceRegion(**{'reducer':reducers, 'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()
        stats_a = after_evi.rename('EVI').reduceRegion(**{'reducer':reducers, 'bestEffort':True, 'scale':30, 'geometry':geometry}).getInfo()

        mean_b = stats_b['EVI_mean']
        mean_a = stats_a['EVI_mean']

        p25_b = stats_b['EVI_p25']
        p25_a = stats_a['EVI_p25']

        p50_b = stats_b['EVI_p50']
        p50_a = stats_a['EVI_p50']

        p75_b = stats_b['EVI_p75']
        p75_a = stats_a['EVI_p75']

        stddev_b = stats_b['EVI_stdDev']
        stddev_a = stats_a['EVI_stdDev']

        min_b = stats_b['EVI_min']
        min_a = stats_a['EVI_min']

        max_b = stats_b['EVI_max']
        max_a = stats_a['EVI_max']

        p25 = p25_a - p25_b * 100
        p50 = p50_a - p50_b * 100
        p75 = p75_a - p50_b * 100
        mean = mean_a - mean_b * 100
        mini = min_a - min_b * 100
        maxi = max_a - max_b * 100
        stddev = stddev_a - stddev_b * 100
  
    except: 
        p25 = None
        p50 = None
        p75 = None
        mean = None
        stddev = None
        mini = None
        maxi = None
    
    return p25, p50, p75, mean, stddev, mini, maxi

def difference(coordinate, date):
    
    point = ee.Geometry.Point(coordinate.getInfo()['features'][0]['geometry']['coordinates'])
    geometry = point.buffer(1000) # buffers point by 1 km

    md = ee.Date(date)
    sd = md.advance(-4, 'week')
    ed = md.advance(4, 'week')

    before_sar = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT").filterBounds(c).filterDate(sd, md).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).limit(1, 'system:time_start', False).first() 
    aafter_sar = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT").filterBounds(c).filterDate(md, ed).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

        
    before_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(c).filterDate(sd, md).sort('CLOUD_COVER').limit(1, 'system:time_start', False).first()
    after_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(c).filterDate(md, ed).sort('CLOUD_COVER').first()
    
    ndvi_p25, ndvi_p50, ndvi_p75, ndvi_mean, ndvi_stddev, ndvi_min, ndvi_max = ndvi(before_landsat, 
                                                                                    after_landsat, 
                                                                                    geometry)  

    evi_p25, evi_p50, evi_p75, evi_mean, evi_stddev, evi_min, evi_max = evi(before_landsat, 
                                                                            after_landsat, 
                                                                            geometry)

    sar_p25, sar_p50, sar_p75, sar_mean, sar_stddev, sar_min, sar_max = sar(before_sar, 
                                                                            
                                                                            after_sar, geometry)

    nbr_p25_, nbr_p50, nbr_p75, nbr_mean, nbr_stddev, nbr_min, nbr_max = nbr(before_landsat, 
                                                                             after_landsat, 
                                                                             geometry)

    
    ar = np.array([ndvi_p25, ndvi_p50, ndvi_p75, ndvi_mean, ndvi_min, ndvi_max, 
                   evi_p25, evi_p50, evi_p75, evi_mean, evi_stddev, evi_min, evi_max, 
                   sar_mean, sar_stddev, sar_max, 
                   nbr_mean, nbr_stddev, nbr_p25_, nbr_p50, nbr_p75, nbr_min])

    return ar

In [12]:
data = pd.read_csv('data.csv')
cols = data.columns
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')
data.drop(columns=['Unnamed: 0', 'SAR_p75', 'SAR_min', 'SAR_p50', 'SAR_p25', 'NDVI_stdDev'], inplace=True)
data.dropna(how='any', axis=0, inplace=True)

In [13]:
x = data.iloc[:, :-2]
y = data.iloc[:, -1]

In [19]:
print(x.shape)
print(y.shape)

(7368, 22)
(7368,)


In [14]:
classifier = Pipeline([('Scale', StandardScaler()), 
                       ('PCA', PCA(n_components = 14, random_state=1)), 
                        ('Classifier', MLPClassifier((400,200), 'relu', 'sgd', 0.01, 50, 'adaptive', 0.1, 250, random_state=0, early_stopping=True, n_iter_no_change=5))])

C:\Users\caleb\miniconda3\envs\ds\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass solver=sgd, alpha=0.01, batch_size=50, learning_rate=adaptive, learning_rate_init=0.1, power_t=250 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [15]:
classifier.fit(x, y)

ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 1. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
classifier.score(x, y)

0.878800217155266

In [25]:
filename = 'finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [22]:
s = pickle.dumps(classifier)
clf = pickle.loads(s)

In [23]:
clf

Pipeline(steps=[('Scale', StandardScaler()),
                ('PCA', PCA(n_components=14, random_state=1)),
                ('Classifier',
                 MLPClassifier(alpha=0.01, batch_size=50, early_stopping=True,
                               hidden_layer_sizes=(400, 200),
                               learning_rate='adaptive', learning_rate_init=0.1,
                               n_iter_no_change=5, power_t=250, random_state=0,
                               solver='sgd'))])